<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/af/examples/binder_hallucination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AfDesign - binder hallucination
For a given protein target and protein binder length, generate/hallucinate a protein binder sequence AlphaFold thinks will bind to the target structure. To do this, we minimize PAE and maximize number of contacts at the interface and within the binder, and we maximize pLDDT of the binder.

**WARNING**
1.   This notebook is in active development and was designed for demonstration purposes only.
2.   Using AfDesign as the only "loss" function for design might be a bad idea, you may find adversarial sequences (aka. sequences that trick AlphaFold).

In [ ]:
#@title install
%%bash
if [ ! -d params ]; then
  pip -q install git+https://github.com/sokrypton/ColabDesign.git
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar | tar x -C params
  for W in openfold_model_ptm_1 openfold_model_ptm_2 openfold_model_no_templ_ptm_1
  do wget -qnc https://files.ipd.uw.edu/krypton/openfold/${W}.npz -P params; done
fi

In [ ]:
#@title #import libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

In [ ]:
#@title # Prep Inputs
pdb = "4N5T" #@param {type:"string"}
chain = "A" #@param {type:"string"}
binder_len =  15#@param {type:"integer"}
hotspot = "" #@param {type:"string"}
if hotspot == "": hotspot = None

x = {"pdb_filename":pdb, "chain":chain, "binder_len":binder_len, "hotspot":hotspot}
if "x_prev" not in dir() or x != x_prev:
  clear_mem()
  model = mk_afdesign_model(protocol="binder")
  x["pdb_filename"] = get_pdb(x["pdb_filename"])
  model.prep_inputs(**x)
  x_prev = x
  print("target length:", model._target_len)
  print("binder length:", model._binder_len)

In [ ]:
#@title #Run AfDesign
#@markdown The default settings below were optimized for helical peptide design.

#@markdown ---
#@markdown ####Weights
#@markdown - Minimizing `pae` or maximizing `plddt` often results in a single helix.
#@markdown To avoid this, we start with a random sequence and instead try to optimize 
#@markdown defined `num`ber of `con`tacts per position. 
plddt = 0.1 #@param ["0.01", "0.1", "0.5", "1.0"] {type:"raw"}
pae = 0.1 #@param ["0.01", "0.1", "0.5", "1.0"] {type:"raw"}
con = 0.1 #@param ["0.01", "0.1", "0.5", "1.0"] {type:"raw"}

weights = {"con":float(con),"pae":float(pae),"plddt":float(plddt)}

#@markdown ####Contact Definition
#@markdown - The contact definition is based on Cb-Cb diststance `cutoff`. To avoid 
#@markdown biasing towards helical contact, only contacts with sequence seperation > 
#@markdown `seqsep` are considered.
seqsep = 0 #@param ["0","5","9"] {type:"raw"}
cutoff = "max" #@param ["8", "14", "max"]
num = "max" #@param ["1", "2", "4", "8", "max"]
binary = True #@param {type:"boolean"}
if cutoff == "max": cutoff = 21.6875
if num == "max": num = binder_len

opt = {"con":{"seqsep":int(seqsep),"cutoff":float(cutoff),"num":int(num),
              "binary":binary,"entropy":True}}
#@markdown ---
#@markdown ####interface Weights
i_pae = 1.0 #@param ["0.01", "0.1", "0.5", "1.0"] {type:"raw"}
i_con = 0.5 #@param ["0.01", "0.1", "0.5", "1.0"] {type:"raw"}

weights.update({"i_pae":float(i_pae),"i_con":float(i_con)})

#@markdown ####interface Contact Definition
cutoff = "max" #@param ["8", "14", "max"]
num = "max" #@param ["1", "2", "4", "8", "max"]
binary = True #@param {type:"boolean"}
if cutoff == "max": cutoff = 21.6875
if num == "max": num = binder_len

opt.update({"i_con":{"cutoff":float(cutoff),"num":int(num),
                     "binary":binary,"entropy":True}})

#@markdown ---
#@markdown Advanced options
seq_init = "None" #@param ["None", "soft_gumbel"]
if seq_init == "None": seq_init = None
# pre-design with gumbel initialization and softmax activation
model.restart(mode=seq_init, opt=opt, weights=weights)
model.design_3stage(100,100,10)

In [ ]:
model.save_pdb(f"{model.protocol}.pdb")
model.plot_pdb()

In [ ]:
HTML(model.animate())

In [ ]:
model.get_seqs()